In [1]:
import pandas as pd 
import os 
from sqlalchemy import create_engine
import dask.dataframe as dd


In [3]:
engine =create_engine('sqlite:///inventory_db.db')

In [7]:
for file in os.listdir('data'):
    if file.endswith('.csv') and file != 'sales.csv':
        df = pd.read_csv('data/' + file)
        print(f"Ingesting {file} with shape {df.shape}")
        ingest_db(df, file[:-4], engine)

Ingesting begin_inventory.csv with shape (206529, 9)
Ingesting end_inventory.csv with shape (224489, 9)
Ingesting purchases.csv with shape (2372474, 16)
Ingesting purchase_prices.csv with shape (12261, 9)
Ingesting vendor_invoice.csv with shape (5543, 10)


In [5]:
def ingest_db(df,table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace',index=False)

In [9]:
big_df = dd.read_csv('data/sales.csv')

In [11]:
for part in big_df.partitions:
    part_df = part.compute()
    part_df.to_sql('sales', con=engine, if_exists='append', index=False)
    